In [1]:
import gradio as gr
import numpy as np
import IPython
from IPython.display import Audio, display
import torch
import librosa, librosa.display
from tqdm import tqdm
from stqdm import stqdm
import json

import sys  
sys.path.insert(0, '../')
from audioldm2 import text_to_audio, build_model, seed_everything, make_batch_for_text_to_audio
from audioldm2.latent_diffusion.modules.diffusionmodules.util import (
    make_ddim_sampling_parameters,
    make_ddim_timesteps,
    noise_like,
    extract_into_tensor,
)
from audioldm2.latent_diffusion.models.ddim import DDIMSampler
from audioldm2.utilities import *
from audioldm2.utilities.audio import *
from audioldm2.utilities.data import *
from audioldm2.utils import default_audioldm_config

from audioldm2.gaverutils import gaver_sounds

from audioldm2.latent_diffusion.modules.attention import SpatialTransformer, CrossAttention

from einops import rearrange, repeat

from scipy.stats import pearsonr


from interfaces.diffusion_helper_qkv import *


import random

import soundfile as sf

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


In [4]:
def get_model(model_name):
    print('Loading model')
    
    latent_diffusion = build_model(model_name=model_name)
    latent_diffusion.latent_t_size = int(duration * latent_t_per_second)

    print('Model loaded')
    return latent_diffusion


In [5]:
model_name = 'audioldm_16k_crossattn_t5'
latent_t_per_second=25.6
sample_rate=16000
duration = 10.0 #Duration is minimum 10 secs. The generated sounds are weird for <10secs
guidance_scale = 3
random_seed = 42
n_candidates = 1
batch_size = 1
ddim_steps = 20

latent_diffusion = get_model(model_name)

Loading model
Loading AudioLDM-2: audioldm_16k_crossattn_t5
Loading model on cuda:0
{'variables': {'sampling_rate': 16000, 'mel_bins': 64, 'latent_embed_dim': 8, 'latent_t_size': 256, 'latent_f_size': 16, 'in_channels': 8, 'optimize_ddpm_parameter': True, 'warmup_steps': 5000}, 'step': {'validation_every_n_epochs': 1, 'save_checkpoint_every_n_steps': 5000, 'max_steps': 1500000, 'save_top_k': 2}, 'preprocessing': {'audio': {'sampling_rate': 16000, 'max_wav_value': 32768, 'duration': 10.24}, 'stft': {'filter_length': 1024, 'hop_length': 160, 'win_length': 1024}, 'mel': {'n_mel_channels': 64, 'mel_fmin': 0, 'mel_fmax': 8000}}, 'augmentation': {'mixup': 0}, 'model': {'target': 'audioldm2.latent_diffusion.models.ddpm.LatentDiffusion', 'params': {'first_stage_config': {'base_learning_rate': 8e-06, 'target': 'audioldm2.latent_encoder.autoencoder.AutoencoderKL', 'params': {'sampling_rate': 16000, 'batchsize': 4, 'monitor': 'val/rec_loss', 'image_key': 'fbank', 'subband': 1, 'embed_dim': 8, 'ti

/home/purnima/anaconda3/envs/audioldm2/lib/python3.8/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/purnima/anaconda3/envs/audioldm2/lib/python3.8/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


DiffusionWrapper has 265.53 M params.


/home/purnima/anaconda3/envs/audioldm2/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Model loaded


In [6]:
def get_word_pairs():
    adjective_noun_pairs = []
    verb_noun_pairs = []

    with open('AudioPairBank/adjective_noun_pairs.txt') as f:
        adjective_noun_pairs_str = f.read()

    with open('AudioPairBank/verb_noun_pairs.txt') as f:
        verb_noun_pairs_str = f.read()

    adjective_noun_pairs = adjective_noun_pairs_str.split("\n")
    verb_noun_pairs = verb_noun_pairs_str.split("\n")

    return np.array(adjective_noun_pairs), np.array(verb_noun_pairs)

adjective_noun_pairs, verb_noun_pairs = get_word_pairs()

In [7]:
len(adjective_noun_pairs), len(verb_noun_pairs)

(759, 359)

In [33]:
def score_wav_text(wav_arr, text, random_seed): #return change in score and pearson corr
    seed_everything(random_seed)
    score_list = []
    for ind, wav in enumerate(wavs):
        wav = torch.from_numpy(wav).unsqueeze(dim=0).cuda()
        score = latent_diffusion.clap.cos_similarity(wav, "The sound of "+text.split(" ")[0])
        # print(ind, score)
        score_list.append(score.detach().cpu())

    change_in_score = torch.abs(score_list[0] - score_list[-1])
    corr = np.abs(pearsonr(score_list, np.arange(0, len(score_list), 1)).statistic)

    return change_in_score, corr

# Reweighting

In [9]:
num_bootstrap = 100
num_bootstrap_choices = 50
random_seed = 1947

In [15]:
def create_samples(num_choices=1, type="adjective"):
    pairs_choices = []
    dir_p = ""
    if type == 'adjective':
        pairs_choices = np.random.choice(adjective_noun_pairs, num_choices, replace=False)
        dir_p = os.path.join('samples_reweighting', 'adjective_noun')
    else:
        pairs_choices = np.random.choice(verb_noun_pairs, num_choices, replace=False)
        dir_p = os.path.join('samples_reweighting', 'verb_noun')

    os.makedirs(dir_p, exist_ok=True)
    for text in sorted(pairs_choices):
        if text is not None and text != "":
            if not os.path.exists(os.path.join(dir_p, text.replace(" ","_")+"_2.wav")):
                print(text)
                describing_word = text.split(" ")[0]
                for i in np.arange(-2,3,1):
                    predicted_wav, _ = sample_diffusion_attention_core(source_text=None, target_text=text, random_seed=random_seed, ddim_steps=20,\
                                                                   latent_diffusion=latent_diffusion,
                                                                   interpolation_level = None,
                                                                   interpolate_terms = ['k','v'],
                                                                   target_selected_word_list=[describing_word], target_value_list=[i],
                                                                   disable_tqdmoutput = True
                                                                   )
            
                    fname = text.replace(" ","_")
                    fname += "_"+str(i)+".wav"
                    
                    sf.write(os.path.join(dir_p, fname), predicted_wav, samplerate=16000) 
            else:
                print(text, 'exists')

## Adjective Noun

In [16]:
create_samples(num_choices=len(adjective_noun_pairs), type="adjective")

annoying air exists
annoying alien
annoying computer
annoying glitch
annoying noise exists
annoying siren
annoying sound
annoying speech exists
annoying voice
annoying woman
calm atmosphere
calm autumn
calm background exists
calm beach
calm bird exists
calm car exists
calm city
calm evening
calm forest
calm mood
calm morning exists
calm nature exists
calm noise exists
calm piano
calm spring
calm street
calm summer exists
calm water
calm waves exists
calm wind exists
calm winter exists
calm wood
calm woods exists
chaotic can exists
chaotic cup exists
chaotic garbage
chaotic glitch
chaotic metal
chaotic noise exists
chaotic water
crazy alien
crazy effects exists
crazy glitch
crazy horror
crazy laughter exists
crazy man
crazy noise exists
crazy speech exists
crazy voice
crazy woman exists
creepy alien
creepy atmosphere
creepy background
creepy bird
creepy child
creepy creatures
creepy death
creepy door
creepy drama exists
creepy effect
creepy engine
creepy fear
creepy ghost exists
creepy 

In [17]:
text = "industrial alert"
describing_word = text.split(" ")[0]
set_weight_for_word(text, ["industrial"], [2], latent_diffusion)

os.path.exists(os.path.join(os.path.join('samples_reweighting', 'adjective_noun'), text.replace(" ","_")+"_2.wav"))

True

## Verb Noun

In [18]:
create_samples(num_choices=len(verb_noun_pairs), type="verb")

accelerating car
accelerating engine
accelerating noise
accelerating race
banging army
banging bird
banging bomb
banging car
banging city
banging construction
banging door
banging fire
banging fireworks
banging footsteps
banging gun
banging metal
banging military
banging noise
banging pot exists
banging rain
banging rocket
banging thunder
banging war
banging wind
banging wood
braking car
braking train
breaking bones
breaking box
breaking cup
breaking egg
breaking footsteps
breaking glass
breaking glasses
breaking ice
breaking noise
breaking snow
breaking waves
breaking window
breaking wood exists
breathing dog
breathing giant
breathing man
breathing monster
breathing voice
breathing woman
burning fire
burning wood
clapping crowd
clapping hand
clapping hands
clapping noise
clapping thunder
clicking bath
clicking car
clicking computer
clicking door
clicking fire
clicking glitch
clicking gun
clicking machine
clicking metal
clicking noise
clicking office
clicking record
clicking tick
click

# Evaluate Adjective Nouns

In [34]:
dir_p = os.path.join('samples_reweighting', 'adjective_noun')

adj_noun_list = []
for f in os.listdir(dir_p):
    adj_noun_list.append(" ".join(f.split(".")[0].split("_")[:-1]))

adj_noun_list = np.unique(np.array(adj_noun_list))

adj_noun_clap_score_change_list = []
adj_noun_perceptual_lin_score_list = []
for adj_noun in adj_noun_list:
    wavs = []
    for i in np.arange(-2,3,1):
        fname = os.path.join(dir_p, adj_noun.replace(" ","_")+"_"+str(i)+".wav")
        audio, _ = librosa.load(fname, sr=16000)
        wavs.append(audio)
    change_in_score, perceptual_lin = score_wav_text(wavs, adj_noun, random_seed=random_seed)
    adj_noun_clap_score_change_list.append(change_in_score)
    adj_noun_perceptual_lin_score_list.append(perceptual_lin)

adj_noun_clap_score_change_list = torch.stack(adj_noun_clap_score_change_list)
adj_noun_perceptual_lin_score_list = np.array(adj_noun_perceptual_lin_score_list)

print("Mean clap score change = ", str(torch.mean(adj_noun_clap_score_change_list).numpy())+"+/-"+str(torch.std(adj_noun_clap_score_change_list).numpy()))
print("Mean Corr = ", str(np.mean(adj_noun_perceptual_lin_score_list))+"+/-"+str(np.std(adj_noun_perceptual_lin_score_list)))
print('-----------------------------')
for ind, adj_noun in enumerate(adj_noun_list):
    print(adj_noun, adj_noun_clap_score_change_list[ind], adj_noun_perceptual_lin_score_list[ind])

Mean clap score change =  0.12382453+/-0.08536095
Mean Corr =  0.663412535321506+/-0.23054602685976075
-----------------------------
annoying air tensor(0.2680) 0.7224125657890523
annoying alien tensor(0.0547) 0.707255951996655
annoying computer tensor(0.1695) 0.8195012264820873
annoying glitch tensor(0.1040) 0.7238571910457755
annoying noise tensor(0.1267) 0.7853411409691264
annoying siren tensor(0.0296) 0.8224777078504921
annoying sound tensor(0.1137) 0.6996293593965724
annoying speech tensor(0.2662) 0.7142777457148533
annoying voice tensor(0.2342) 0.7442714893397089
annoying woman tensor(0.1077) 0.39684236034482506
calm atmosphere tensor(0.0888) 0.7497637028121121
calm autumn tensor(0.1953) 0.7148796699875701
calm background tensor(0.1603) 0.6849999964492436
calm beach tensor(0.1781) 0.7004241849816266
calm bird tensor(0.1720) 0.7420559575885508
calm car tensor(0.0698) 0.8520858947600487
calm city tensor(0.3591) 0.8624328192578838
calm evening tensor(0.1642) 0.8488904749949366
calm 

# Evaluate Verb Noun

In [35]:
dir_p = os.path.join('samples_reweighting', 'verb_noun')

verb_noun_list = []
for f in os.listdir(dir_p):
    verb_noun_list.append(" ".join(f.split(".")[0].split("_")[:-1]))

verb_noun_list = np.unique(np.array(verb_noun_list))

verb_noun_clap_score_change_list = []
verb_noun_perceptual_lin_score_list = []
for verb_noun in verb_noun_list:
    wavs = []
    for i in np.arange(-2,3,1):
        fname = os.path.join(dir_p, verb_noun.replace(" ","_")+"_"+str(i)+".wav")
        audio, _ = librosa.load(fname, sr=16000)
        wavs.append(audio)
    change_in_score, perceptual_lin = score_wav_text(wavs, verb_noun, random_seed=random_seed)
    verb_noun_clap_score_change_list.append(change_in_score)
    verb_noun_perceptual_lin_score_list.append(perceptual_lin)

verb_noun_clap_score_change_list = torch.stack(verb_noun_clap_score_change_list)
verb_noun_perceptual_lin_score_list = np.array(verb_noun_perceptual_lin_score_list)

print("Mean clap score change = ", str(torch.mean(verb_noun_clap_score_change_list).numpy())+"+/-"+str(torch.std(verb_noun_clap_score_change_list).numpy()))
print("Mean Corr = ", str(np.mean(verb_noun_perceptual_lin_score_list))+"+/-"+str(np.std(verb_noun_perceptual_lin_score_list)))
print('-----------------------------')
for ind, verb_noun in enumerate(verb_noun_list):
    print(verb_noun, verb_noun_clap_score_change_list[ind], verb_noun_perceptual_lin_score_list[ind])

Mean clap score change =  0.17013133+/-0.0828925
Mean Corr =  0.6925575031977739+/-0.19157755132528595
-----------------------------
accelerating car tensor(0.2952) 0.9332541486790712
accelerating engine tensor(0.3174) 0.9913724492274156
accelerating noise tensor(0.2587) 0.9235453241744132
accelerating race tensor(0.3133) 0.9068057930173956
banging army tensor(0.2458) 0.4395720718914517
banging bird tensor(0.3460) 0.7934650908993541
banging bomb tensor(0.1045) 0.32707174490966795
banging car tensor(0.2454) 0.2829160507608659
banging city tensor(0.2920) 0.8141419378546447
banging construction tensor(0.2817) 0.950170568390519
banging door tensor(0.1960) 0.2715012186051141
banging fire tensor(0.2218) 0.4956405987442374
banging fireworks tensor(0.1982) 0.9490305894387502
banging footsteps tensor(0.2558) 0.905870357226976
banging gun tensor(0.2120) 0.8242966891153278
banging metal tensor(0.1646) 0.4922569202823669
banging military tensor(0.2381) 0.6207279377196027
banging noise tensor(0.222

In [29]:
# dir_p = os.path.join('samples_reweighting', 'verb_noun')

# f="ringing_church_bell_-2.wav"
# " ".join(f.split(".")[0].split("_")[:-1])

# Bootstrapped Adjective Noun scores

In [44]:
bootstrap_clap_score_change_list = []
bootstrap_perceptual_lin_score_list = []
for bootstrap_ind in range(num_bootstrap):
    adj_noun_list_ind_choices = np.random.choice(np.arange(len(adj_noun_list)), num_bootstrap_choices, replace=False)
    adj_noun_clap_score_change_sublist = adj_noun_clap_score_change_list[adj_noun_list_ind_choices]
    adj_noun_perceptual_lin_score_sublist = adj_noun_perceptual_lin_score_list[adj_noun_list_ind_choices]
    adj_noun_sublist = adj_noun_list[adj_noun_list_ind_choices]

    bootstrap_clap_score_change_list.append(torch.mean(adj_noun_clap_score_change_sublist))
    bootstrap_perceptual_lin_score_list.append(np.mean(adj_noun_perceptual_lin_score_sublist))

bootstrap_clap_score_change_list = torch.stack(bootstrap_clap_score_change_list)
bootstrap_perceptual_lin_score_list = np.array(bootstrap_perceptual_lin_score_list)
print(torch.mean(bootstrap_clap_score_change_list), torch.std(bootstrap_clap_score_change_list), \
      np.mean(bootstrap_perceptual_lin_score_list), np.std(bootstrap_perceptual_lin_score_list))

tensor(0.1268) tensor(0.0115) 0.6675429844587235 0.02891685630595013


# Bootstrapped Verb Noun scores

In [47]:
bootstrap_clap_score_change_list = []
bootstrap_perceptual_lin_score_list = []
for bootstrap_ind in range(num_bootstrap):
    verb_noun_list_ind_choices = np.random.choice(np.arange(len(verb_noun_list)), num_bootstrap_choices, replace=False)
    verb_noun_clap_score_change_sublist = verb_noun_clap_score_change_list[verb_noun_list_ind_choices]
    verb_noun_perceptual_lin_score_sublist = verb_noun_perceptual_lin_score_list[verb_noun_list_ind_choices]
    verb_noun_sublist = verb_noun_list[verb_noun_list_ind_choices]

    bootstrap_clap_score_change_list.append(torch.mean(verb_noun_clap_score_change_sublist))
    bootstrap_perceptual_lin_score_list.append(np.mean(verb_noun_perceptual_lin_score_sublist))

bootstrap_clap_score_change_list = torch.stack(bootstrap_clap_score_change_list)
bootstrap_perceptual_lin_score_list = np.array(bootstrap_perceptual_lin_score_list)
print(torch.mean(bootstrap_clap_score_change_list), torch.std(bootstrap_clap_score_change_list),\
      np.mean(bootstrap_perceptual_lin_score_list), np.std(bootstrap_perceptual_lin_score_list))

tensor(0.1715) tensor(0.0108) 0.6979243160021261 0.023738342466140365


(100, 50)